In [0]:
# Definición de Widgets para parámetros de entrada
# El formato es: dbutils.widgets.text("nombre_variable", "valor_por_defecto", "Etiqueta Visual")

dbutils.widgets.text("aws_access_key", "", "1. AWS Access Key")
dbutils.widgets.text("aws_secret_key", "", "2. AWS Secret Key")
dbutils.widgets.text("s3_bucket_name", "nombre-de-tu-bucket", "3. Nombre del Bucket")

# Widget de lista desplegable para el entorno (esto da un toque Senior)
dbutils.widgets.dropdown("environment", "dev", ["dev", "qa", "prod"], "4. Entorno")

In [0]:
# Recuperar los valores ingresados por el usuario
ACCESS_KEY = dbutils.widgets.get("aws_access_key")
SECRET_KEY = dbutils.widgets.get("aws_secret_key")
BUCKET_NAME = dbutils.widgets.get("s3_bucket_name")
env = dbutils.widgets.get("environment")

# Validación de seguridad
if not access_key or not secret_key:
    dbutils.notebook.exit("Error: Debes ingresar las credenciales de AWS en los widgets superiores.")
else:
    print(f"✅ Configuración cargada exitosamente para el entorno: {env}")

In [0]:
import boto3
import os
import pyarrow.parquet as pq

# --- 2. CONFIGURACIÓN DEL PROCESO ---
REGION = "us-east-2"
NOMBRE_TABLA = "nyc_taxi_2025_enero" 
FILE_KEY = "fhvhv_tripdata_2025-01.parquet"

# Ruta temporal local
local_path = f"/tmp/{FILE_KEY.split('/')[-1]}"
try:
    # --- PASO 1: Descargar de S3 a Disco Local (/tmp) ---
    print(f"1. Descargando a {local_path} ...")
    if not os.path.exists(local_path):
        s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY, region_name=REGION)
        s3.download_file(BUCKET_NAME, FILE_KEY, local_path)
        print("✅ Descarga completada.")
    else:
        print("✅ El archivo ya estaba descargado.")

    # --- PASO 2: Procesamiento por Lotes (Batching) ---
    print(f"2. Escribiendo en la tabla administrada: '{NOMBRE_TABLA}'...")
    
    # Abrimos el archivo con PyArrow
    parquet_file = pq.ParquetFile(local_path)
    
    # Lotes pequeños para que no falle la memoria
    total_batches = parquet_file.num_row_groups
    print(f"   Total de grupos a procesar: {total_batches}")

    for i in range(total_batches):
        # 1. Leer pedacito con Pandas
        batch = parquet_file.read_row_group(i)
        pdf = batch.to_pandas()
        
        # 2. Convertir a Spark
        df_chunk = spark.createDataFrame(pdf)
        
        # 3. GUARDAR COMO TABLA (Aquí está la magia)
        if i == 0:
            # La primera vez: SOBREESCRIBIMOS la tabla
            df_chunk.write.mode("overwrite").saveAsTable(NOMBRE_TABLA)
        else:
            # Las siguientes veces: AGREGAMOS (Append) a la tabla existente
            df_chunk.write.mode("append").saveAsTable(NOMBRE_TABLA)
        
        # Feedback visual cada 10 lotes para no llenar la consola
        if i % 10 == 0:
            print(f"   ✅ Lote {i+1}/{total_batches} guardado en tabla.")

    print(f"\n🎉 ¡VICTORIA! Datos guardados en la tabla '{NOMBRE_TABLA}'")
    
    # --- PASO 3: Verificar Resultado ---
    # Ahora puedes consultar la tabla usando SQL estándar
    print("3. Consultando la tabla final...")
    df_final = spark.table(NOMBRE_TABLA)
    print(f"   Total de filas en la tabla: {df_final.count()}")
    display(df_final.limit(5))

    # Limpieza
    if os.path.exists(local_path):
        os.remove(local_path)

except Exception as e:
    print(f"❌ Error: {str(e)}")

In [0]:
%sql
SELECT * FROM nyc_taxi_2025_enero LIMIT 10;

In [0]:
%sql
select count(1) FROM nyc_taxi_2025_enero